<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/pavelzw/GEFCom14-S-comparison/blob/main/gefcom14-s-nnqf.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/pavelzw/GEFCom14-S-comparison/blob/main/gefcom14-s-nnqf.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://minhaskamal.github.io/DownGit/#/home?url=https://github.com/pavelzw/GEFCom14-S-comparison/blob/main/gefcom14-s-deepar.ipynb&fileName=gefcom14-s-nnqf"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Download data

In [1]:
!echo "Downloading GEFCom14-S..."
!rm -R data > /dev/null
!wget -O gefcom14.zip https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip?dl=0
!unzip gefcom14 > /dev/null
!rm gefcom14.zip > /dev/null
!unzip GEFCom2014\ Data/GEFCom2014-S_V2.zip > /dev/null
!rm -R GEFCom2014\ Data > /dev/null
!mv Solar data > /dev/null
!echo "------------------------------"
!echo "Downloaded GEFCom14-S in data/"

# Prepare data

In [2]:
import numpy as np
import pandas as pd

task = 2
# only use surface solar radiation (169), surface thermal radiation (175) and top net solar radiation (178)
predictors = pd.read_csv(f'data/Task {task}/predictors{task}.csv', parse_dates=['TIMESTAMP'])\
    [['ZONEID', 'TIMESTAMP', 'VAR169', 'VAR175', 'VAR178']].set_index('TIMESTAMP')
train = pd.read_csv(f'data/Task {task}/train{task}.csv', parse_dates=['TIMESTAMP']).set_index('TIMESTAMP')
benchmark = pd.read_csv(f"data/Task {task-1}/benchmark{'0' + str(task-1) if task-1 < 10 else task}.csv",
                        parse_dates=['TIMESTAMP']).set_index('TIMESTAMP')

solar_plants = [train[train['ZONEID'] == i][['POWER']].rename({'POWER': f'ZONEID {i}'}, axis='columns')
                for i in [1,2,3]]
train_data = pd.concat(solar_plants, axis=1)

predictors_categories = [predictors[predictors['ZONEID'] == i][['VAR169', 'VAR175', 'VAR178']]
                             .rename({'VAR169': f'SURFACE SOLAR RADIATION {i}',
                                      'VAR175': f'SURFACE THERMAL RADIATION {i}',
                                      'VAR178': f'TOP NET SOLAR RADIATION {i}'}, axis='columns')
                         for i in [1,2,3]]
predictor_data = pd.concat(predictors_categories, axis=1)[:'2013-05-01 00:00']

In [3]:
predictor1 = predictor_data[['SURFACE SOLAR RADIATION 1', 'SURFACE THERMAL RADIATION 1', 'TOP NET SOLAR RADIATION 1']]

train_data1 = train_data[['ZONEID 1']]

In [4]:
x_input_df = predictor1
x_input = np.array(x_input_df)
y_output = np.array(train_data1)

In [5]:
from sklearn.neighbors import NearestNeighbors

minkowski_dist = 2
x_neighbors = NearestNeighbors(n_neighbors=50, algorithm='auto', p=minkowski_dist).fit(x_input)
dist, indx = x_neighbors.kneighbors(x_input)

In [6]:
y_neighbors = y_output[indx[0, :]].T
for i in range(1, np.size(indx, 0)):
    values_to_add = y_output[indx[i, :]].T
    y_neighbors = np.vstack([y_neighbors, values_to_add])

In [7]:
yq_output = np.quantile(y_neighbors, q=[p / 100 for p in range(1, 100)], axis=1).T
yq_output_df = pd.DataFrame(yq_output, index=x_input_df.index)\
    .rename(columns={p: str((p+1)/100) for p in range(99)})
yq_output_df

,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,...,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2012-04-01 01:00:00,0.325074,0.486365,0.505211,0.517995,0.532077,0.546274,0.562346,0.578679,0.601452,0.625481,...,0.841795,0.842737,0.843715,0.844878,0.846878,0.854888,0.863836,0.883342,0.902090,0.903032
2012-04-01 02:00:00,0.085936,0.099756,0.192001,0.287582,0.366721,0.444399,0.471193,0.490887,0.495180,0.496468,...,0.821442,0.824112,0.826797,0.829562,0.832760,0.839074,0.845972,0.859447,0.873092,0.890745
2012-04-01 03:00:00,0.098879,0.197759,0.270035,0.341179,0.347247,0.347529,0.375238,0.406774,0.412869,0.414000,...,0.758160,0.758569,0.758915,0.758947,0.759597,0.764686,0.770987,0.790933,0.810664,0.825364
2012-04-01 04:00:00,0.071335,0.142542,0.172504,0.200710,0.204397,0.205905,0.206696,0.207387,0.212651,0.218808,...,0.615276,0.616312,0.617308,0.618093,0.619305,0.623577,0.628623,0.642381,0.657579,0.706642
2012-04-01 05:00:00,0.026924,0.030003,0.034708,0.039482,0.040679,0.041559,0.042687,0.043849,0.044564,0.045192,...,0.503051,0.517688,0.531741,0.542797,0.552677,0.554122,0.558800,0.599853,0.639619,0.649137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-04-30 20:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.211071,0.214431,0.245367,0.417621,0.569544,0.575763,0.582156,0.590512,0.599101,0.613204
2013-04-30 21:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.542821,0.556955,0.569797,0.576017,0.582497,0.590853,0.600308,0.622138,0.646018,0.718073
2013-04-30 22:00:00,0.000000,0.000000,0.000000,0.000000,0.000288,0.000603,0.000751,0.000877,0.001292,0.001763,...,0.581615,0.582306,0.584136,0.591800,0.599837,0.610548,0.621292,0.632412,0.646018,0.718073


# Train the linear regression model (ANN10)

## Prepare data for regression model

In [8]:
import pandas as pd
from sklearn import preprocessing

# normalize predictor values
x = predictor1.values
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(x)
predictor1 = pd.DataFrame(scaled)
predictor1

,0,1,2
0,0.070457,0.010104,0.057922
1,0.152360,0.047458,0.142628
2,0.227996,0.084522,0.224650
3,0.285150,0.121753,0.290287
4,0.322960,0.161768,0.338952
...,...,...,...
9475,0.220287,0.683490,0.256298
9476,0.220471,0.720201,0.256733
9477,0.226896,0.758322,0.267201
9478,0.244268,0.796864,0.291180


In [9]:
h = 24
h1 = 24

data = predictor1
x_pieces = []
for i in range(h1):
    piece = data.drop(range(i)).reset_index().drop('index', axis=1)
    x_pieces.insert(0, piece)

x = pd.concat(x_pieces, axis=1)
x.index = x.index.map(lambda x: x + h1 - 1)
x = x[:-h+1]
x

,0,1,2,0,1,2,0,1,2,0,...,2,0,1,2,0,1,2,0,1,2
23,0.450364,0.837689,0.506044,0.406165,0.796552,0.454506,0.376154,0.758559,0.417812,0.360530,...,0.290287,0.227996,0.084522,0.224650,0.152360,0.047458,0.142628,0.070457,0.010104,0.057922
24,0.045105,0.013434,0.039247,0.450364,0.837689,0.506044,0.406165,0.796552,0.454506,0.376154,...,0.338952,0.285150,0.121753,0.290287,0.227996,0.084522,0.224650,0.152360,0.047458,0.142628
25,0.092515,0.053876,0.098001,0.045105,0.013434,0.039247,0.450364,0.837689,0.506044,0.406165,...,0.368936,0.322960,0.161768,0.338952,0.285150,0.121753,0.290287,0.227996,0.084522,0.224650
26,0.145630,0.095325,0.160508,0.092515,0.053876,0.098001,0.045105,0.013434,0.039247,0.450364,...,0.388876,0.342790,0.200918,0.368936,0.322960,0.161768,0.338952,0.285150,0.121753,0.290287
27,0.198880,0.135820,0.222171,0.145630,0.095325,0.160508,0.092515,0.053876,0.098001,0.045105,...,0.394023,0.356316,0.238150,0.388876,0.342790,0.200918,0.368936,0.322960,0.161768,0.338952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9475,0.220287,0.683490,0.256298,0.220287,0.646925,0.256298,0.220287,0.613121,0.256298,0.220287,...,0.387337,0.329628,0.833101,0.358668,0.309107,0.795727,0.332274,0.305069,0.757640,0.324127
9476,0.220471,0.720201,0.256733,0.220287,0.683490,0.256298,0.220287,0.646925,0.256298,0.220287,...,0.017545,0.349089,0.872224,0.387337,0.329628,0.833101,0.358668,0.309107,0.795727,0.332274
9477,0.226896,0.758322,0.267201,0.220471,0.720201,0.256733,0.220287,0.683490,0.256298,0.220287,...,0.063724,0.020428,0.014910,0.017545,0.349089,0.872224,0.387337,0.329628,0.833101,0.358668
9478,0.244268,0.796864,0.291180,0.226896,0.758322,0.267201,0.220471,0.720201,0.256733,0.220287,...,0.119693,0.056263,0.054627,0.063724,0.020428,0.014910,0.017545,0.349089,0.872224,0.387337


In [10]:
from sklearn.neural_network import MLPRegressor
from tqdm import tqdm

prediction = pd.DataFrame()

for p in tqdm(range(1, 100)):
    y = yq_output_df.reset_index().drop('TIMESTAMP', axis=1).drop(range(h1-1))[str(p/100)]
    model = MLPRegressor(hidden_layer_sizes=(6,))

    trained = model.fit(x[:-24*30], y[:-24*30])
    prediction.insert(p-1, str(p/100), trained.predict(x[-24*30:]))

prediction

100%|██████████| 99/99 [00:58<00:00,  1.69it/s]


,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,...,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99
0,0.167383,0.213278,0.237100,0.302447,0.323161,0.287237,0.347237,0.351798,0.392957,0.337612,...,0.807347,0.846042,0.787106,0.832573,0.852729,0.882909,0.808327,0.866491,0.904169,0.825671
1,0.234482,0.280707,0.268995,0.353297,0.352449,0.365478,0.380751,0.394960,0.422626,0.413334,...,0.803782,0.828693,0.820339,0.851787,0.911148,0.878614,0.831767,0.866993,0.886908,0.825671
2,0.239233,0.248735,0.251229,0.316796,0.325106,0.300717,0.337155,0.381028,0.395107,0.380344,...,0.747710,0.771480,0.749459,0.793470,0.862186,0.835834,0.803109,0.824687,0.822201,0.813143
3,0.164125,0.197023,0.182277,0.214218,0.229559,0.214144,0.252581,0.282923,0.336122,0.326846,...,0.654380,0.666673,0.719377,0.709829,0.781214,0.727541,0.732931,0.728029,0.763533,0.707076
4,0.055321,0.080528,0.076193,0.134463,0.091388,0.143478,0.136330,0.120015,0.201728,0.189246,...,0.535942,0.536634,0.561117,0.611603,0.656572,0.661266,0.631701,0.569262,0.669207,0.652675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.018350,0.004713,-0.023331,-0.022603,-0.022901,-0.011089,-0.028234,-0.036396,-0.013093,0.020701,...,0.214661,0.249407,0.231498,0.318176,0.337180,0.343603,0.351199,0.374901,0.353494,0.396412
716,0.011441,0.004713,-0.041438,0.018533,0.005430,-0.014529,-0.005863,-0.098098,-0.008055,-0.011548,...,0.299445,0.297547,0.226758,0.404083,0.430365,0.380261,0.401462,0.412199,0.441784,0.415209
717,0.030472,0.004713,-0.001376,0.034183,-0.016034,0.002141,-0.017697,-0.051888,-0.009368,-0.036219,...,0.385998,0.392216,0.385719,0.522726,0.523380,0.531310,0.561529,0.527206,0.538442,0.534621
718,0.021852,0.004713,-0.033894,0.000751,0.044730,-0.070975,-0.003298,-0.035085,-0.020736,0.016223,...,0.510391,0.536718,0.510260,0.648294,0.554280,0.629662,0.551755,0.647816,0.610791,0.630213


# Postprocessing: Remove quantile crossing

In [11]:
prediction['0.01'] = np.maximum(prediction['0.01'], 0)
for p in range(2, 100):
    prediction[str(p/100)] = np.maximum(prediction[str((p-1)/100)], prediction[str(p/100)])
prediction

,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,...,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99
0,0.167383,0.213278,0.237100,0.302447,0.323161,0.323161,0.347237,0.351798,0.392957,0.392957,...,0.811886,0.846042,0.846042,0.846042,0.852729,0.882909,0.882909,0.882909,0.904169,0.904169
1,0.234482,0.280707,0.280707,0.353297,0.353297,0.365478,0.380751,0.394960,0.422626,0.422626,...,0.828829,0.828829,0.828829,0.851787,0.911148,0.911148,0.911148,0.911148,0.911148,0.911148
2,0.239233,0.248735,0.251229,0.316796,0.325106,0.325106,0.337155,0.381028,0.395107,0.395107,...,0.813062,0.813062,0.813062,0.813062,0.862186,0.862186,0.862186,0.862186,0.862186,0.862186
3,0.164125,0.197023,0.197023,0.214218,0.229559,0.229559,0.252581,0.282923,0.336122,0.336122,...,0.712231,0.712231,0.719377,0.719377,0.781214,0.781214,0.781214,0.781214,0.781214,0.781214
4,0.055321,0.080528,0.080528,0.134463,0.134463,0.143478,0.143478,0.143478,0.201728,0.201728,...,0.587270,0.587270,0.587270,0.611603,0.656572,0.661266,0.661266,0.661266,0.669207,0.669207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.018350,0.018350,0.018350,0.018350,0.018350,0.018350,0.018350,0.018350,0.018350,0.020701,...,0.258827,0.258827,0.258827,0.318176,0.337180,0.343603,0.351199,0.374901,0.374901,0.396412
716,0.011441,0.011441,0.011441,0.018533,0.018533,0.018533,0.018533,0.018533,0.018533,0.018533,...,0.308432,0.308432,0.308432,0.404083,0.430365,0.430365,0.430365,0.430365,0.441784,0.441784
717,0.030472,0.030472,0.030472,0.034183,0.034183,0.034183,0.034183,0.034183,0.034183,0.034183,...,0.439467,0.439467,0.439467,0.522726,0.523380,0.531310,0.561529,0.561529,0.561529,0.561529
718,0.021852,0.021852,0.021852,0.021852,0.044730,0.044730,0.044730,0.044730,0.044730,0.044730,...,0.569276,0.569276,0.569276,0.648294,0.648294,0.648294,0.648294,0.648294,0.648294,0.648294


# Evaluate Loss

The loss function is the pinnball loss:
$$ L(q_a, y) = \begin{cases}
    (1-\frac{a}{100})(q_a - y), &\text{if } y < q_a \\
    \frac{a}{100}(y-q_a), &\text{if } y \geq q_a.
\end{cases} $$

The score is then averaged over all target quantiles for all time periods over the forecast horizon and for all zones.

In [12]:
def pinnball_loss(actual, prediction):
    actual = actual[...,None]

    percentiles = np.empty((actual.shape[0], 99))
    for i in range(1, 100):
        percentiles[:, i-1] = i
    loss = np.where(actual < prediction,
                    (1 - percentiles / 100) * (prediction - actual),
                    percentiles / 100 * (actual - prediction))
    return loss

In [13]:
test_ts = np.squeeze(train_data1['2013-04-01 01:00':].values)
loss = np.mean(pinnball_loss(test_ts, yq_output_df['2013-04-01 01:00':]))

print(f"Loss of task {task} only nearest neighbors: {round(loss, 5)}")

Loss of task 2 only nearest neighbors: 0.02366


In [14]:
loss = np.mean(pinnball_loss(test_ts, prediction))

print(f"Loss of task {task} with NNQF: {round(loss, 5)}")

Loss of task 2 with NNQF: 0.04499
